In [1]:
%load_ext autoreload
%autoreload 2

import formatOrtho
import pyhamPipeline

import pyham
from pyoma.browser import db 
import numpy as np
from tables import *
import re
from ete3 import Tree
import pickle
import tempfile
import functools
import config

h5file = open_file(config.omadirLaurent + 'OmaServer.h5', mode="r")
dbObj = db.Database(h5file)
omaIdObj = db.OmaIdMapper(dbObj)

/home/laurent/anaconda3/lib/python3.6/site-packages/tables/leaf.py:396: PerformanceWarning: The Leaf ``/Protein/_i_Entries/OmaHOG/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


In [2]:
import fix_species_tree



In [ ]:
%load_ext autoreload
%autoreload 2


import pyham
from pyoma.browser import db 
import numpy as np
from tables import *
import re
from ete3 import Tree
import pickle
import tempfile
import functools
import config

import pyhamPipeline
import profileGen
import format_files

from datasketch import MinHashLSH

# with open( config.working_dir + "speciestree.nwk" , 'r') as treefile:
#     species_tree = treefile.read()
# t = Tree(species_tree, format =1 )
# print(t)
# #rewrite IDs and reload tree
# #
# species_tree = pyham.utils.get_newick_string(working_dir + "speciestree_hack.nwk", type="nwk")


#open up OMA
h5file = open_file(config.omadirLaurent + 'OmaServer.h5', mode="r") 
#setup db objects
dbObj = db.Database(h5file)
omaIdObj = db.OmaIdMapper(dbObj)

# corrects species tree and replacement dictionary for orthoXML files

dic, tree = format_files.create_species_tree(h5file, omaIdObj)


#temporary, 
mat_list = []
dico_list = []
lsh = MinHashLSH()

#load Fam
for fam in pyhamPipeline.yieldFamilies(h5file):
#generate pyham object
    if fam > 0 and fam < 500:
        try:
            print(fam)
            if fam % 100 == 0:
                print(fam)
            #generate treemap profile
            treemap_fam = pyhamPipeline.get_hamTree(fam, dbObj, tree, dic)
            # generate matrix of hash
            dico = profileGen.Tree2Hashes(treemap_fam, fam, lsh)
            dico_list.append(dico)

            # generate taxa index
            #taxaIndex, taxaIndexReverse = profileGen.generateTaxaIndex(species_tree)

            # generate matrix of 1 and 0 for each biological event
            #profileGen.Tree2mat(treemap_fam, taxaIndex)
            #mat_list.append(mat)

#         except ValueError:
#             print("{} can't be used by pyHam: minimum two genomes are required.".format(fam))
#         except KeyError as kerr:
#             print("KeyError: {} at {}".format(kerr, fam))
        except:
             print('breaking at fam: {}'.format(fam))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/laurent/anaconda3/lib/python3.6/site-packages/tables/leaf.py:396: PerformanceWarning: The Leaf ``/Protein/_i_Entries/OmaHOG/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275


In [52]:
Missing species

Methanomethylophilus alvus (strain Mx1201)
Methanomassiliicoccus intestinalis (strain Issoire-Mx1)
Thermoplasmatales archaeon (strain BRNA1)
Natrialba asiatica (strain ATCC 700177 / DSM 12278 / JCM 9576 / FERM P-10747 / NBRC 102637 / 172P1)

Other errors

'2 node(s) founded for the species name: Methanococcus_maripaludis'

breaking at fam: 13309
breaking at fam: 13314
breaking at fam: 13317
breaking at fam: 13321
breaking at fam: 13322
breaking at fam: 13324
breaking at fam: 13325
breaking at fam: 13326
breaking at fam: 13327
breaking at fam: 13328
breaking at fam: 13329
breaking at fam: 13333
breaking at fam: 13335
breaking at fam: 13337
breaking at fam: 13339
breaking at fam: 13342
breaking at fam: 13343
breaking at fam: 13344
breaking at fam: 13346
breaking at fam: 13347
breaking at fam: 13351
breaking at fam: 13352
breaking at fam: 13353
breaking at fam: 13356
breaking at fam: 13358
breaking at fam: 13361
breaking at fam: 13366
breaking at fam: 13367
breaking at fam: 13368
breaking at fam: 13369
breaking at fam: 13371
breaking at fam: 13388
breaking at fam: 13394
breaking at fam: 13398
breaking at fam: 13399
breaking at fam: 13400
breaking at fam: 13404
breaking at fam: 13406
breaking at fam: 13411
breaking at fam: 13415
    
    KeyError: ''

{'Korarchaeum cryptofilum (strain OPF8)': 'Korarchaeum_cryptofilum__strain_OPF8_', 'Nanoarchaeum equitans (strain Kin4-M)': 'Nanoarchaeum_equitans__strain_Kin4-M_', 'Caldisericum exile (strain DSM 21853 / NBRC 104410 / AZM16c01)': 'Caldisericum_exile__strain_DSM_21853_/_NBRC_104410_/_AZM16c01_', 'Desulfurispirillum indicum (strain ATCC BAA-1389 / S5)': 'Desulfurispirillum_indicum__strain_ATCC_BAA-1389_/_S5_', 'Cloacimonas acidaminovorans (strain Evry)': 'Cloacimonas_acidaminovorans__strain_Evry_', 'Fibrobacter succinogenes (strain ATCC 19169 / S85)': 'Fibrobacter_succinogenes__strain_ATCC_19169_/_S85_', 'Gemmatimonas aurantiaca (strain T-27 / DSM 14586 / JCM 11422 / NBRC 100505)': 'Gemmatimonas_aurantiaca__strain_T-27_/_DSM_14586_/_JCM_11422_/_NBRC_100505_', 'Thermobaculum terrenum (strain ATCC BAA-798 / YNP1)': 'Thermobaculum_terrenum__strain_ATCC_BAA-798_/_YNP1_', 'Guillardia theta': 'Guillardia_theta', 'Giardia intestinalis': 'Giardia_intestinalis', 'Emiliania huxleyi': 'Emiliania_h

In [ ]:
dico_list

In [134]:
dico_list[2]['3-loss']

In [135]:
result = lsh.query(dico_list[2]['3-loss'])


In [136]:
result

['3-loss', '6-loss', '3-loss-duplication', '6-loss-duplication']

In [23]:
from ete3 import NCBITaxa

ncbi = NCBITaxa()

bigtree = ncbi.get_descendant_taxa('cellular organisms', collapse_subspecies=False, return_tree=True)


In [24]:
type(bigtree)

ete3.phylo.phylotree.PhyloNode

In [15]:
tree = bigtree.write(format=1)

In [7]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

In [8]:
tree = ncbi.get_topology(genome_ids_list)

In [2]:
import pandas as pd
genomeTab = h5file.root.Genome
genome_df = pd.DataFrame(genomeTab.read())

In [5]:
genome_ids_list = pd.DataFrame(h5file.root.Genome.read())["NCBITaxonId"].tolist()

In [88]:
for node in tree.traverse():
    node.name = node.sci_name
    
    #print(num)
    #taxonId_SciName[num]

In [21]:
full_dataFrame = pd.DataFrame(h5file.root.Genome.read())
restricted_dataFrame = full_dataFrame[["NCBITaxonId","SciName"]]

In [62]:
restricted_dataFrame.loc[restricted_dataFrame["NCBITaxonId"] == 131567, "SciName"].values[0].decode()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [29]:
print(tree)

((((((((Desulfotomaculum_nigrificans_CO-1-SRB:1,Desulfotomaculum_acetoxidans_DSM_771:1,Desulfotomaculum_reducens_MI-1:1,Desulfotomaculum_kuznetsovii_DSM_6115:1,Desulfotomaculum_ruminis_DSM_2154:1)Desulfotomaculum:1,((Desulfitobacterium_hafniense_DCB-2:1,Desulfitobacterium_hafniense_Y51:1)Desulfitobacterium_hafniense:1,Desulfitobacterium_dichloroeliminans_LMG_P-21439:1)Desulfitobacterium:1,(Desulfosporosinus_meridiei_DSM_13257:1,Desulfosporosinus_orientis_DSM_765:1,Desulfosporosinus_acidiphilus_SJ4:1)Desulfosporosinus:1,Pelotomaculum_thermopropionicum_SI:1,Candidatus_Desulforudis_audaxviator_MP104C:1,Thermincola_potens_JR:1,Syntrophobotulus_glycolicus_DSM_8271:1)Peptococcaceae:1,((Ruminococcus_champanellensis_18P13___JCM_17042:1,Ruminococcus_albus_7___DSM_20455:1)Ruminococcus:1,((Ruminiclostridium_thermocellum_ATCC_27405:1,Ruminiclostridium_thermocellum_DSM_1313:1)Ruminiclostridium_thermocellum:1,_Clostridium__cellulolyticum_H10:1,_Clostridium__clariflavum_DSM_19732:1)Ruminiclostridium:

In [61]:
restricted_dataFrame

,NCBITaxonId,SciName
0,374847,b'Korarchaeum cryptofilum (strain OPF8)'
1,1072681,b'Haloredivivus sp. (strain G17)'
2,889948,b'Nanosalina sp. (strain J07AB43)'
3,889962,b'Nanosalinarum sp. (strain J07AB56)'
4,666510,b'Acidilobus saccharovorans (strain DSM 16705 ...
5,1056495,b'Caldisphaera lagunensis (strain DSM 15908 / ...
6,272557,b'Aeropyrum pernix (strain ATCC 700893 / DSM 1...
7,490899,b'Desulfurococcus kamchatkensis (strain DSM 18...
8,765177,b'Desulfurococcus mucosus (strain ATCC 35584 /...
9,453591,b'Ignicoccus hospitalis (strain KIN4/I / DSM 1...


In [43]:
taxonId_SciName = {}
for genome in h5file.root.Genome.read():
    taxonId_SciName[genome[0]] = genome[5].decode()
    
taxonId_SciName

{374847: 'Korarchaeum cryptofilum (strain OPF8)',
 1072681: 'Haloredivivus sp. (strain G17)',
 889948: 'Nanosalina sp. (strain J07AB43)',
 889962: 'Nanosalinarum sp. (strain J07AB56)',
 666510: 'Acidilobus saccharovorans (strain DSM 16705 / JCM 18335 / VKM B-2471 / 345-15)',
 1056495: 'Caldisphaera lagunensis (strain DSM 15908 / JCM 11604 / IC-154)',
 272557: 'Aeropyrum pernix (strain ATCC 700893 / DSM 11879 / JCM 9820 / NBRC 100138 / K1)',
 490899: 'Desulfurococcus kamchatkensis (strain DSM 18924 / JCM 16383 / VKM B-2413 / 1221n)',
 765177: 'Desulfurococcus mucosus (strain ATCC 35584 / DSM 2162 / JCM 9187 / O7/1)',
 453591: 'Ignicoccus hospitalis (strain KIN4/I / DSM 18386 / JCM 14125)',
 583356: 'Ignisphaera aggregans (strain DSM 17230 / JCM 13409 / AQ1.S1)',
 591019: 'Staphylothermus hellenicus (strain DSM 12710 / JCM 10830 / BK20S6-10-b1 / P8)',
 399550: 'Staphylothermus marinus (strain ATCC 43588 / DSM 3639 / JCM 9404 / F1)',
 1184251: 'Thermogladius cellulolyticus (strain 1633)',

In [65]:
import fix_species_tree

dic, tree = fix_species_tree.create_species_tree(h5file, omaIdObj)

In [66]:
dic

{'STAA3': 'Staphylococcus_aureus__strain_USA300_',
 'ECO57': 'Escherichia_coli_O157_H7',
 'BUCAI': 'Buchnera_aphidicola_subsp__Acyrthosiphon_pisum__strain_APS_',
 'CHLPN': 'Chlamydia_pneumoniae',
 'Caldisericum exile (strain DSM 21853 / NBRC 104410 / AZM16c01)': 'Caldisericum_exile__strain_DSM_21853_/_NBRC_104410_/_AZM16c01_',
 'Desulfurispirillum indicum (strain ATCC BAA-1389 / S5)': 'Desulfurispirillum_indicum__strain_ATCC_BAA-1389_/_S5_',
 'Emiliania huxleyi': 'Emiliania_huxleyi',
 'Guillardia theta': 'Guillardia_theta',
 'Naegleria gruberi': 'Naegleria_gruberi',
 'Thermobaculum terrenum (strain ATCC BAA-798 / YNP1)': 'Thermobaculum_terrenum__strain_ATCC_BAA-798_/_YNP1_',
 'Coprothermobacter proteolyticus (strain ATCC 35245 / DSM 5265 / BT)': 'Coprothermobacter_proteolyticus__strain_ATCC_35245_/_DSM_5265_/_BT_',
 'Cloacimonas acidaminovorans (strain Evry)': 'Cloacimonas_acidaminovorans__strain_Evry_',
 'Gemmatimonas aurantiaca (strain T-27 / DSM 14586 / JCM 11422 / NBRC 100505)': 'G

In [67]:
tree

"((((((((Desulfotomaculum_nigrificans__strain_DSM_14880_/_VKM_B-2319_/_CO-1-SRB_:1,Desulfotomaculum_acetoxidans__strain_ATCC_49208_/_DSM_771_/_VKM_B-1644_:1,Desulfotomaculum_reducens__strain_MI-1_:1,Desulfotomaculum_kuznetsovii__strain_DSM_6115_/_VKM_B-1805_/_17_:1,Desulfotomaculum_ruminis__strain_ATCC_23193_/_DSM_2154_/_NCIB_8452_/_DL_:1)Desulfotomaculum:1,((Desulfitobacterium_hafniense__strain_DCB-2_/_DSM_10664_:1,Desulfitobacterium_hafniense__strain_Y51_:1)Desulfitobacterium hafniense:1,Desulfitobacterium_dichloroeliminans__strain_LMG_P-21439_/_DCA1_:1)Desulfitobacterium:1,(Desulfosporosinus_meridiei__strain_ATCC_BAA-275_/_DSM_13257_/_NCIMB_13706_/_S10_:1,Desulfosporosinus_orientis__strain_ATCC_19365_/_DSM_765_/_NCIMB_8382_/_VKM_B-1628_:1,Desulfosporosinus_acidiphilus__strain_DSM_22704_/_JCM_16185_/_SJ4_:1)Desulfosporosinus:1,Pelotomaculum_thermopropionicum__strain_DSM_13744_/_JCM_10971_/_SI_:1,Desulforudis_audaxviator__strain_MP104C_:1,Thermincola_potens__strain_JR_:1,Syntrophobotu

In [ ]:
import pyhamPipeline

for fam in pyhamPipeline.yieldFamilies(h5file):
    for fam > 200 and fam >= 1000:
    print(fam)
    treemap_fam = pyhamPipeline.get_hamTree(fam, dbObj, tree, dic)



In [56]:
dic

{'STAA3': 'Staphylococcus_aureus__strain_USA300_',
 'ECO57': 'Escherichia_coli_O157_H7',
 'BUCAI': 'Buchnera_aphidicola_subsp__Acyrthosiphon_pisum__strain_APS_',
 'CHLPN': 'Chlamydia_pneumoniae',
 'Terrabacteria group': 'Terrabacteria_group',
 'FCB group': 'FCB_group',
 'PVC group': 'PVC_group',
 'Caldisericum exile _strain DSM 21853 / NBRC 104410 / AZM16c01_': 'Caldisericum_exile__strain_DSM_21853_/_NBRC_104410_/_AZM16c01_',
 'Desulfurispirillum indicum _strain ATCC BAA-1389 / S5_': 'Desulfurispirillum_indicum__strain_ATCC_BAA-1389_/_S5_',
 'Giardia intestinalis': 'Giardia_intestinalis',
 'Emiliania huxleyi': 'Emiliania_huxleyi',
 'Guillardia theta': 'Guillardia_theta',
 'Naegleria gruberi': 'Naegleria_gruberi',
 'TACK group': 'TACK_group',
 'DPANN group': 'DPANN_group',
 'Thermobaculum terrenum _strain ATCC BAA-798 / YNP1_': 'Thermobaculum_terrenum__strain_ATCC_BAA-798_/_YNP1_',
 'Coprothermobacter proteolyticus _strain ATCC 35245 / DSM 5265 / BT_': 'Coprothermobacter_proteolyticus__